In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [5]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [6]:
import json
from typing import Any, Dict

import boto3
import yaml

from core.task_processor import TaskProcessor
from core.process_registry import ProcessRegistry
from core.task_publisher import TaskPublisher

from core.processes.client_bank_statement import ClientBankStatementProcess

In [7]:
ENV = "dev"
tenant_id = "alfa"

with open(f"../config/tenants/{tenant_id}/{ENV}.yaml", "r") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {})
dynamodb = boto3.resource(
    "dynamodb",
    region_name=dynamodb_config.get("region"),
)
messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))

sqs_config = tenant_config.get("sqs", {})
sqs = boto3.client("sqs", region_name=sqs_config.get("region"))
queue_url = sqs_config.get("tasks_url")

In [8]:
task_publisher = TaskPublisher(sqs, queue_url)

processor = TaskProcessor(
    tenant_config=tenant_config,
    messages_table=messages_table,
    processes_table=processes_table,
    task_publisher=task_publisher,
)

In [ ]:
max_messages = 10
wait_time_seconds = 5

resp = sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=max_messages,
    WaitTimeSeconds=wait_time_seconds,
)

messages = resp.get("Messages", [])

print(f"Received {len(messages)} messages")

for msg in messages:
    body = msg["Body"]
    print("Raw body:", body)

    processed, remaining = processor.process(body)

    if processed:
        print("✅ Task processed immediately")
    else:
        if remaining < 0:
            print("🗑️ Task dropped (stale)")
        else:
            delay = min(int(remaining), 60)
            print(f"⏳ Task needs to wait {remaining}s → requeue with delay={delay}s")

            sqs.send_message(
                QueueUrl=queue_url,
                MessageBody=body,
                DelaySeconds=delay,
            )

    sqs.delete_message(
        QueueUrl=queue_url,
        ReceiptHandle=msg["ReceiptHandle"],
    )
    print("Deleted message:", msg["MessageId"])

Received 1 messages
Raw body: {"task_type": "READ_INCOMING_WHATSAPP_MESSAGE", "agent_type": "ACCOUNTING_ASSISTANT", "process_type": "READ_INCOMING_WHATSAPP_MESSAGE", "context_key": "whatsapp:525571969848", "payload": {"text": "Tambi\u00e9n me gusta cantar"}, "debounce_policy": {"type": "messages_idle", "min_idle_seconds": 60}, "timestamp_iso": "2025-12-13T04:30:05", "timestamp_epoch": 1765600205}
